*By: B830459\
Date: 20/1/2023*

## Program Explanation: 

The program returns an output in the form of a dataframe. The dataframe includes information about a specific week, from a specific module, which are both specified by the user. The infromation in mind is about all the events that have happend during the week and their *Time*, *Room*, *Type* and *Average Attendance* percentage. 	

In order for the above to be executed the user, must specify the fucntion **module_attendance ( )** parameters.\
Those are a dictionary with all the modules names as keys and their formatted dataframe form from **neat_table( )** as values. Alongside the module name and week that are desired to be examined should be specified as a string type.
The module name should be taken from the dictionary keys. Whilist the week's should be specified under the format "W" + the week number in mind, such as "W5" for week 5.

The function will highlight the average attendance for each event, given the set rules:
- If average attendance is below 50% - highlight in red
- If average attendance is above 50% but below 70%  - highlight in yellow
- If average attendance is above 70% - highlight in green

In the case that an invalid module name or week has been entered the program will notify the user that such module or week does not exist amongst the provided data. 

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from IPython.display import clear_output
from tkinter import *

## Database Imports

In [2]:
def import_from_db(df_name):
    """
    
    Imports SQL tables from CWDatabase.db
    in the form of pandas dataframe.
    
    The function requries to be provided with:
    df_name -> the SQL table name as a 
    string type
    
    
    """
    conn = sqlite3.connect("CWDatabase.db")
    
    table_to_get = "SELECT * FROM " + df_name
    import_sql = pd.read_sql(table_to_get, conn)
    
    conn.close()
    
    return import_sql

## Extracing information from imported database tables

In [3]:
def neat_table(df, timetable):
    """
     
    Creates a dataframe, provided:
    
    df -> the students attendance SQL 
    table name as a string type
    timetable -> the modules timetable SQL 
    table name as a string type
    
    The newly created dataframe, sets the 
    "Timetable" type table from the 
    database and its respective values as 
    index for the "Records" table.
    Each individual students attendance 
    is represented under a column, 
    matching the respective multi index 
    for the specific class/lecture.
    
    """
    # Modifying the student records table, so that it includes a session id KEY 
    indexed = import_from_db(df).set_index("student_id")
    records = indexed.T
    empty = []
    for i in records.index:
        get_after = "Session_ID:_"
        result = i.split(get_after, 1)
        empty.append(int(result[1]))
    
    records["sess_id"] = empty
    
    # Importing the timetable 
    schedule = import_from_db(timetable)
    
    # Merging the updated records DF with the timetable DF
    complete_module = pd.merge(schedule, records, on = "sess_id")
    
    # Indexes that have to be set of the concated DF
    index = ["sem", "week", "sess_id", "date", "time", "type", "room"]
    
    # Setting those indexes to work 
    complete_module.set_index(index, inplace = True)
    
    # Fillining all Nans to 0
    complete_module = complete_module.fillna(0)
    
    return complete_module

## Styling of end output

In [4]:
def bcolor_code(value):
    """
    
    Sets background colors 
    of table cells value.
    
    """
    
    if value < 50:
        colour = "#FFB3B3"
    elif value < 70:
        colour = "#FFD966"
    else:
        colour = "#77DD77"
    styler = f"background-color:{colour}"
    
    return styler

## Main

In [5]:
def module_attendance(func, entry_df, entry_week):
    """
    
    Provides a dataframe that lists out all of the 
    events(classes/lectures) from a week for a specified module 
    and the average attendance percentage for each of it. 
    
    The function requries to be provided with:
    func -> a dictionary containing as its values, 
    dataframes for the individual modules, being processed
    under the neat_table( ) function
    entry_df -> a key value from the dictionary 
    (module name) in a string type 
    entry_week -> the week that the end user would 
    like to observe in a string type

    entry_week must follow the format upper case
    followed by the week number e.g. "W5".
    
    
    """
    # Check if module does exist in the provided dictionary
    while True:
        if entry_df in func.keys(): 
            df = func[entry_df]
            break

        else:
            print("You have not selected a module to review")
            print("Try again")
            return
    
    # From the provided module above, check if the provided weeks does exist 
    # extract weeks present in the module
    rawdf = df.reset_index()
    rawdf = rawdf["week"].tolist()
    weeks_in_module = list(set(rawdf))
    
    # Check if week exist
    while True:
        if entry_week in weeks_in_module:
            break
        else:
            print("You have not selected a week to review")
            print("Try again")
            return
        
    # Calculations
    actual_att = df.groupby(["week","sess_id", "time", "room", "type"]).sum().sum(1)
    ideal_att = df.groupby(["week","sess_id", "time", "room", "type"]).count().sum(1)
    
    # Return new data frame for output containing fullset of weeks
    output = pd.DataFrame((actual_att / ideal_att * 100).round(2)).droplevel(1)
    output.columns = ["Attendance %"]
    
    # Printout final result based on student_id input
    output = output.loc[entry_week]
    
    # Formating end output visable to user
    output.index.set_names(["Time", "Room", "Type"], inplace=True)
    output = output.style.format({"Attendance %": "{:.2f}"}).applymap(bcolor_code)
    
    print("Attendance record for " + entry_df.upper() + " Week " + entry_week[1:] + ":")
    display(output)

## Related to GUI Tkinter Dropdown Menus

In [6]:
def week_getter(func):
    """
    
    A new dictionary is created from a provided one.
    The new dictionary has as its:
    keys -> module names
    values -> the names of every week available in the 
              module
              
    The function requries to be provided with:
    
    func -> a dictionary containing as its values, 
    DataFrames for the individual modules, being processed
    under the neat_table( ) function
    
    """
    week_dict = {}
    for i in func:
        week_list = []
        for x in func[i].reset_index()["week"].to_list():
            if x not in week_list:
                week_list.append(x)
    
        week_dict[i] = week_list
        
    return week_dict

## Testing

In [7]:
def import_modules():
    """
    
    The function intiliazes all of the modules information 
    into "neat" dataframes, by making use of the 
    neat_table( ) function.
    
    The function is used predomentatly as a 
    dictionary to access multiple modules 
    attendances. 
    
    """
    
    coa111 = neat_table("dfcoa111", "sesscoa111")
    coa122 = neat_table("dfcoa122", "sesscoa122")
    
    return locals()

In [8]:
module_attendance(import_modules(), "coa122", "W4")

Attendance record for COA122 Week 4:


,,,Attendance %
Time,Room,Type,
14:00 - 16:00,CC012,Lecture,74.37
09:00 - 11:00,N001...,Computer Lab,25.63
11:00 - 13:00,N001...,Computer Lab,24.62
12:00 - 13:00,CC011,Personal Best,74.37


In [9]:
module_attendance(import_modules(), "coa111", "W2")

Attendance record for COA111 Week 2:


,,,Attendance %
Time,Room,Type,
10:00 - 11:00,SMB014,Lecture,69.35
14:00 - 15:00,CC011,Lecture,80.90


In [10]:
module_attendance(import_modules(), "invalid_module", "W4")

You have not selected a module to review
Try again


In [11]:
module_attendance(import_modules(), "coa111", "invalid_week")

You have not selected a week to review
Try again


In [12]:
week_getter(import_modules())

{'coa111': ['W1', 'W2', 'W3', 'W4', 'W5'],
 'coa122': ['W1', 'W2', 'W3', 'W4', 'W5', 'W6']}